Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
# Imports libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set_theme()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the csv files from part 2 with Foursquare and Yelp EDA
cb = pd.read_csv('../data/cb_montreal_stations.csv')
fsq = pd.read_csv('../data/fsq_bar_features.csv')
yelp = pd.read_csv('../data/yelp_bar_features.csv')

In [ ]:
fsq.sort_values(by='name')

In [ ]:
yelp.sort_values(by='name')

In [ ]:
fsq['name'].nunique()

In [ ]:
yelp['name'].nunique()

A few observations here:

* There are more unique bar names in the 'fsq' dataframe.
* The category of bars from 'fsq' is better categorized, because Foursquare already has a taxonomy of well-define category. We used search string "bars" for categories in the Yelp API call, thus it's not as well-organized. We also added 
* The 'distance' column in Yelp has values greater than 1000m, although we set the radius=1000 only.

For the reasons above, we'll use bar information from Foursquare as the base of our clean data; then add additional information such as 'review_count', 'rating', 'price' from Yelp.

In [ ]:
# Merge to add 'review_count', 'rating', 'price' columns from the 'yelp' to 'fsq' dataframe
fsq = fsq.merge(yelp[['name', 'review_count', 'rating', 'price']].drop_duplicates(subset=['name']), on='name', how='left')

In [ ]:
# Check again the new 'fsq' dataframe after adding 'review_count', 'rating', 'price' from Yelp
fsq

Now, we just need to merge the new 'fsq' with the Citybikes data in the 'cb' dataframe.

In [ ]:
# Merge 'cb' and 'fsq' based on 'cb_latitude' and 'cb_longitude' in 'cb' and 'latitude' and 'longitude' in 'fsq'
merged_all_df = pd.merge(cb, fsq, left_on=['cb_latitude', 'cb_longitude'], right_on=['latitude', 'longitude'], how='inner')

# Drop the repeated 'latitude' and 'longitude' columns from the merged DataFrame
merged_all_df = merged_all_df.drop(['latitude', 'longitude'], axis=1)

merged_all_df

In [ ]:
merged_all_df.shape

In [ ]:
# Save our master dataframe 'merged_all_df' results to csv
merged_all_df.to_csv('../data/merged_all_df.csv', index=False)

### Exploring the data

In [ ]:
merged_all_df.info()

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
merged_all_df.describe()

In [ ]:
# Add a new column 'unique_name_count' to store the count of unique bars per citybike station in Montreal
merged_all_df['cb_name_count'] = merged_all_df.groupby('cb_station_name')['name'].transform('nunique')

In [ ]:
merged_all_df.head()

In [ ]:
# Check where the missing values are located
merged_all_df.isnull().sum()

* Treat missing values in 'postcode':

In [ ]:
yelp[['name','postcode']].drop_duplicates()

In [ ]:
# Create a mapping of 'name' to 'postcode' from the Yelp dataframe above. It has a bigger dataset than Foursquare
unique_yelp_postcode_mapping = yelp[['name', 'postcode']].drop_duplicates().set_index('name')['postcode'].to_dict()

# Use the mapping to fill missing 'postcode' values in merged_all_df
merged_all_df['postcode'].fillna(merged_all_df['name'].map(unique_yelp_postcode_mapping), inplace=True)

In [ ]:
merged_all_df['postcode'].isnull().sum()

In [ ]:
# Fill the rest of 176 missing values in 'postcode' with 'Postcode Unavailable'
merged_all_df['postcode'].fillna('Unavailable', inplace=True)

* Treat missing values in 'review_count', 'rating':

In [ ]:
# Create 2 graphs to visualize the distribution of  'review_count' and 'rating':
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

# Plot 0
axes[0].hist(merged_all_df['review_count'])
axes[0].set_title('Histogram of review_count')
axes[0].set_xlabel('review_count')
axes[0].set_ylabel('counts')

# Plot 1
axes[1].hist(merged_all_df['rating'], color='green')
axes[1].set_title('Histogram of rating')
axes[1].set_xlabel('rating')
axes[1].set_ylabel('counts')
axes[1].set_xlim(left=0)

plt.show()

In [ ]:
# As the distributions of these two columns are skewed, we will fill in the missing values with the median.
merged_all_df['review_count'].fillna(merged_all_df['review_count'].median(), inplace=True)
merged_all_df['rating'].fillna(merged_all_df['rating'].median(), inplace=True)

* Treat missing values in 'price':

In [ ]:
merged_all_df['price'].value_counts()

In [ ]:
# Calculate the percentage of each price category
(merged_all_df['price'].value_counts() / merged_all_df['price'].notnull().sum()) * 100

In [ ]:
# The '$$' value is most populated with > 80%, we'll use it to fill in missing values in this column
merged_all_df['price'].fillna('$$', inplace=True)

According to this paper at: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0252157#:~:text=Yelp%20use%20dollar%20signs%20(%24),higher%20level%20means%20more%20expensive

"Yelp uses dollar signs ($) to categorize approximate cost per person for a meal in a restaurant. In particular, “$” means under $10; “$$” means “$11-$30”; “$$$” means “$31-$60”; and “$$$$” means “above $61”. "

We can reassign the funky dollar sign values in 'price' column into a categorical system that takes 1 to 4 as price levels. Higher level means more expensive.

In [ ]:
def assign_price_level(price):
    if price == '$':
        return 1
    elif price == '$$':
        return 2
    elif price == '$$$':
        return 3
    else:
        return 4
    
merged_all_df['price_level'] = merged_all_df['price'].apply(assign_price_level)

In [ ]:
# Drop 'price' as we already indicated the same thing in 'price_level'
merged_all_df = merged_all_df.drop('price', axis=1)

In [ ]:
merged_all_df

In [ ]:
# Final check for missing values
merged_all_df.isnull().sum()

In [ ]:
merged_all_df.shape

In [ ]:
# Drop duplicates (if any) after treating n/a, but there aren't any because cb_station_id is unique
merged_all_df.drop_duplicates(inplace=True)

### Visualizing the data

In [ ]:
merged_all_df.info()

* Exploring numeric columns:

In [ ]:
numeric_cols = merged_all_df.select_dtypes(include=['number']).columns
numeric_cols

In [ ]:
numeric_merge_df = merged_all_df[numeric_cols]
numeric_merge_df

In [ ]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows, cols, i+1)
        dataframe[feature].hist(bins=20, ax=ax, facecolor='Midnightblue')
        ax.set_title(feature + " Distribution",color='DarkRed')
        ax.set_yscale('log')
    fig.tight_layout()  
    plt.show()
    
draw_histograms(numeric_merge_df,numeric_merge_df.columns,8,4)

There are still bar locations that are further out than 1Km from Montreal bike stations. The distribution of 'cb_bike_num' is skewed with outliers.

As per characteristic of bars (of numeric type except for 'price_level'): 'distance' looks somewhat like a uniform distribution. The distribution of 'review_count' looks skewed, 'rating' has a pretty good shape of normal distribution without outliers. 

We are dealing with outliers, and we can also test the normal distribution of some of those numeric columns.

In [ ]:
# Enforce the condition 'distance'values must be <= 1000 from the bike stations
merged_all_df = merged_all_df[merged_all_df['distance'] <= 1000]

* Treating outliers: We won't take into accounts outliers in 'cb_latitude' and 'cb_longitude' since we want to include all bike stations in Montreal.

In [ ]:
# Percentage of rows with outliers where 'cb_bike_num' >50
len(merged_all_df[merged_all_df['cb_bike_num'] <= 50])/len(merged_all_df)

In [ ]:
# Percentage of rows with outliers where 'review_count' >300
len(merged_all_df[merged_all_df['review_count'] <= 300])/len(merged_all_df)

We'll lose around 1% of our data (~60 rows) for each outlier removal which is quite ok.

In [ ]:
# Remove extreme outliers where 'cb_bike_num' >50
merged_all_df = merged_all_df[merged_all_df['cb_bike_num'] <= 50]

In [ ]:
# Remove extreme outliers where 'review_count' >300
merged_all_df = merged_all_df[merged_all_df['review_count'] <= 300]

In [ ]:
merged_all_df.shape

* Perform Shapiro-Wilk's normality test on 'cb_bike_num', 'cb_name_count', 'distance', 'review_count', 'rating':
- H<sub>0</sub>: data is normally distributed
- H<sub>a</sub>: data is not normally distributed


In [ ]:
stat, p = stats.shapiro(merged_all_df['cb_bike_num'])
print(p)

In [ ]:
stat, p = stats.shapiro(merged_all_df['cb_name_count'])
print(p)

In [ ]:
stat, p = stats.shapiro(merged_all_df['distance'])
print(p)

In [ ]:
stat, p = stats.shapiro(merged_all_df['review_count'])
print(p)

In [ ]:
stat, p = stats.shapiro(merged_all_df['rating'])
print(p)

Since p<<0.05, we reject the null hypothesis that the data in these columns are normally distributed. We'll have to perform data normalization at a latter step.

* Exploring categorical columns:

Which station has the most bars close by?

In [ ]:
merged_all_df['cb_station_id'].nunique()

In [ ]:
merged_all_df['cb_station_name'].nunique()

In [ ]:
stations_barnum_sorted = merged_all_df.groupby(['cb_station_id', 'cb_station_name'])['cb_name_count'].mean().sort_values(ascending=False).reset_index()
stations_barnum_sorted

In [ ]:
# The majority of numbar of bars associated per station is 10 bars on average
stations_barnum_sorted['cb_name_count'].describe()

In [ ]:
fig=plt.figure(figsize=(4,2))
sns.boxplot(x=stations_barnum_sorted['cb_name_count'])
plt.show()

In [ ]:
# The top stations with the most number of bars will be mostly those with 10 bars nearby
top_barnum_stations = stations_barnum_sorted[stations_barnum_sorted['cb_name_count']==10]
top_barnum_stations

Is there any relationship between the bike station name and the bar postcode and categories?

In [ ]:
stations_postcode = merged_all_df[['cb_station_name','postcode']].drop_duplicates()
stations_postcode

In [ ]:
stations_postcode['postcode_prefix'] = stations_postcode['postcode'].str[:3]
stations_postcode = stations_postcode[stations_postcode['postcode_prefix'] != 'Una'].drop('postcode', axis=1).drop_duplicates()
stations_postcode

It's unclear if there's any relationship between the bike station name and the bar postcode and categories at this point, because of the large number of unique combinations. Visualization won't be the best option to carry out EDA for our categorical columns overall. To keep things simpler, it's better to just use numerical columns for prediction in our model in part 4, and not going forward with encoding categorical data.

Recall in the previous section of data visualization, all of our numeric columns have p value = 0 or very small in the normality test. However, we won't apply data transformation to convert those numeric columns into normal distribution within our scope.

* Correlation matrix:

In [ ]:
merged_all_df.corr()

In [ ]:
sns.heatmap(merged_all_df.corr(), annot = True, cmap="Blues")

From this plot we can see that it seems like there may be a very small linear correlation between our target variable ('cb_bike_num') and these bar characteristics:

* Minor positive correlation with 'review_count' (0.18), 'price level' (0.065)
* Minor negative correlation with 'distance'(-0.11), 'rating' (-0.019)

A correlation near zero indicates the variables are not linearly related. Thus, we can exclude the variables with near zero correlation. Thus, the dependent variables kept in our model to predict our our target variable ('cb_bike_num') are 'review_count', 'distance'.

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Our scope for this SQLite3 database in Python for part 3:

* Create connection and access data from three API results (dataframes) that we have from part 2 into 3 tables
* Use SQL queries to access data from those 3 tables
* Join data from three multiple tables into a database

In [ ]:
# Import libraries and modules
import sqlite3
from sqlite3 import Error

* Create connection and access data from three API results (dataframes) that we have from part 2 into 3 tables

In [ ]:

def create_connection(path):
    """ Create a database connection to a SQLite database """
    connection = None
    try:
        connection = sqlite3.connect(path)
        print('Connection to SQLite DB successful')
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [ ]:
# Create an empty database named 'merged_all_sql.db' in '../data/' if not exists
connection = create_connection('../data/merged_all_sql.db')

In [ ]:
# Create a new set of dataframes again for citybikes foursquare and yelp data
cb_sql_df = pd.read_csv('../data/cb_montreal_stations.csv')
fsq_sql_df = pd.read_csv('../data/fsq_bar_features.csv')
yelp_sql_df = pd.read_csv('../data/yelp_bar_features.csv')

In [ ]:
# Write DataFrame to SQLite tables: 'citybikes', 'foursquare', 'yelp' without index
cb_sql_df.to_sql('citybikes', connection, if_exists='replace', index='False')
fsq_sql_df.to_sql('foursquare', connection, if_exists='replace', index='False')
yelp_sql_df.to_sql('yelp', connection, if_exists='replace', index='False')

* Use SQL queries to access data from those 3 tables

Look at the data before and after the join to validate your data.

In [ ]:
# Create a function execute_read_query()
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
# Query data from the 'citybikes' table
select_citybikes = "SELECT * from citybikes"
cb_rows = execute_read_query(connection, select_citybikes)

for cb_row in cb_rows:
    print(cb_row)

In [ ]:
# Query data from the 'foursquare' table
select_foursquare = "SELECT * from foursquare"
fsq_rows = execute_read_query(connection, select_foursquare)

for fsq_row in fsq_rows:
    print(fsq_row)

In [ ]:
# Query data from the 'yelp' table
select_yelp = "SELECT * from yelp"
yelp_rows = execute_read_query(connection, select_yelp)

for yelp_row in yelp_rows:
    print(yelp_row)

* Join data from three multiple tables into a database

In [ ]:
# Apply the same concept as in the pandas joining tables
# Add 'review_count', 'rating', 'price' columns from the 'yelp' to 'fsq' dataframe

left_join_foursquare_yelp = '''
CREATE VIEW left_join_foursquare_yelp
AS 
SELECT foursquare.*
FROM foursquare
LEFT JOIN (
    SELECT DISTINCT name, review_count, rating, price
    FROM yelp
) As tbl ON foursquare.name = tbl.name;
'''

fsq_yelp_rows = execute_read_query(connection, left_join_foursquare_yelp)

In [ ]:
# Create a view that combines the 'left_join_foursquare_yelp' view and 'citybikes' table

foursquare_yelp_citybikes = '''
CREATE VIEW merge_foursquare_yelp_citybikes
AS 
SELECT *
FROM left_join_foursquare_yelp
INNER JOIN citybikes AS cb
ON cb.cb_latitude = left_join_foursquare_yelp.latitude
AND cb.cb_longitude = left_join_foursquare_yelp.longitude;
'''

foursquare_yelp_citybikes = execute_read_query(connection, foursquare_yelp_citybikes)